<div align="center">
  <img src="../assets/images/hackathon.png" alt="Holistic AI Hackathon Logo" width="600"/>
</div>

**Event**: [hackathon.holisticai.com](https://hackathon.holisticai.com)

---


# Tutorial 1: Building ReAct Agents with LangGraph

**Learn to build intelligent agents using LangGraph's prebuilt ReAct pattern**

## What You'll Learn

1. **Understand**: What is a ReAct agent and how it differs from direct LLM calls
2. **Build**: Your first ReAct agent using LangGraph's official implementation
3. **Add Tools**: Integrate Valyu AI search for real-time information
4. **Compare**: Agent performance with and without tools

## What is ReAct?

**ReAct** = **Reasoning** + **Acting**

- Traditional approach (2022): LLM generates explicit "Thought:", "Action:", "Observation:" steps in text
- Modern approach (2023-2025): LLMs use **function calling** - structured tool execution built into the model API
- This tutorial uses the **modern approach** with LangGraph's prebuilt agent

### Modern ReAct Pattern Flow

```
User Input
    ↓
LLM Reasoning (implicit)
    ↓
Decision: Need tools?
    ├─→ YES: Call tool → Get results → Loop back
    └─→ NO: Generate final answer → End
```

---

## Step 0: Install Dependencies

Run this cell to install all required packages in this notebook.

In [1]:
# Install required packages
!pip install -q \
    langgraph\
    langchain-core\
    langchain-openai\
    langchain-valyu \
    python-dotenv \
    requests

print("✅ All dependencies installed!")

✅ All dependencies installed!


## Step 1: Setup Environment

**Recommended:**
Set up Holistic AI Bedrock Proxy API credentials in `.env`:

```bash
HOLISTIC_AI_TEAM_ID=tutorials_api
HOLISTIC_AI_API_TOKEN=your-token-here
```

[API Guide](../assets/api-guide.pdf)

**Alternative:**
If you prefer OpenAI:

```bash
OPENAI_API_KEY=your-openai-api-key-here
```

**Optional:**
```bash
VALYU_API_KEY=your-valyu-api-key-here
```

**Note:** The tutorial uses Holistic AI Bedrock by default (recommended). OpenAI is optional - to use it, call get_chat_model(model_name, use_openai=True) and set OPENAI_API_KEY.

In [2]:
import os
from pathlib import Path
from dotenv import load_dotenv

# ============================================
# OPTION 1: Set API keys directly (Quick Start)
# ============================================
# Uncomment and set your keys here:
# Recommended: Holistic AI Bedrock
# os.environ["HOLISTIC_AI_TEAM_ID"] = "tutorials_api"
# os.environ["HOLISTIC_AI_API_TOKEN"] = "your-token-here"
# Alternative: OpenAI
# os.environ["OPENAI_API_KEY"] = "your-openai-key-here"
# Optional: Valyu
# os.environ["VALYU_API_KEY"] = "your-valyu-key-here"

# ============================================
# OPTION 2: Load from .env file (Recommended)
# ============================================
# Try to load from .env file in parent directory
env_path = Path('../.env')
if env_path.exists():
    load_dotenv(env_path)
    print("📄 Loaded configuration from .env file")
else:
    print("⚠️  No .env file found - using environment variables or hardcoded keys")

# ============================================
# Verify API keys are set
# ============================================
print("\n🔑 API Key Status:")
if os.getenv('HOLISTIC_AI_TEAM_ID') and os.getenv('HOLISTIC_AI_API_TOKEN'):
    print("  ✅ Holistic AI Bedrock credentials loaded (will use Bedrock)")
elif os.getenv('OPENAI_API_KEY'):
    print("  ⚠️  OpenAI API key loaded (Bedrock credentials not set)")
    print("     💡 Tip: Set HOLISTIC_AI_TEAM_ID and HOLISTIC_AI_API_TOKEN to use Bedrock (recommended)")
else:
    print("  ⚠️  No API keys found")
    print("     Set Holistic AI Bedrock credentials (recommended) or OpenAI key")

if os.getenv('VALYU_API_KEY'):
    key_preview = os.getenv('VALYU_API_KEY')[:10] + "..."
    print(f"  ✅ Valyu API key loaded: {key_preview}")
else:
    print("  ⚠️  Valyu API key not found - search tool will not work")

print("\n📁 Working directory:", Path.cwd())

# ============================================
# Import Holistic AI Bedrock helper function
# ============================================
# Import from core module (recommended)
import sys
try:
    sys.path.insert(0, '../core')
    from react_agent.holistic_ai_bedrock import HolisticAIBedrockChat, get_chat_model
    print("\n✅ Holistic AI Bedrock helper function loaded")
except ImportError:
    print("\n⚠️  Could not import from core - will use OpenAI only")
    print("   Make sure core/react_agent/holistic_ai_bedrock.py exists")

# Import official packages
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage

print("\n✅ All imports successful!")


📄 Loaded configuration from .env file

🔑 API Key Status:
  ✅ Holistic AI Bedrock credentials loaded (will use Bedrock)
  ✅ Valyu API key loaded: 2Qx4tyw7ey...

📁 Working directory: /Users/zekunwu/Desktop/hackthon-2026/tutorials

✅ Holistic AI Bedrock helper function loaded

✅ All imports successful!


### Model Helper Function

This helper function uses by default the best available model:

**Recommended:**
- **Holistic AI Bedrock** - Uses Claude, Llama, Nova models via Bedrock Proxy
- Set `HOLISTIC_AI_TEAM_ID` and `HOLISTIC_AI_API_TOKEN` in `.env`
- [API Guide](../assets/api-guide.pdf)

**Alternative:**
- **OpenAI** - Uses OpenAI models (optional)

You can use model names like:
- `claude-3-5-sonnet` → Uses Holistic AI Bedrock (Claude Sonnet) - **Recommended**
- `claude-3-5-haiku` → Uses Holistic AI Bedrock (Claude Haiku - faster)
- `gpt-5-mini` → Uses OpenAI (optional, requires use_openai=True)

## Step 2: Understanding the Difference

### Direct LLM Call vs ReAct Agent

| Aspect | Direct LLM Call | ReAct Agent |
|--------|----------------|-------------|
| **Pattern** | One-shot: question → answer | Reasoning + Acting in a loop |
| **Tools** | ❌ No tool access | ✅ Can use search, APIs, databases |
| **Multi-step** | ❌ Single response | ✅ Can break down complex tasks |
| **Self-correction** | ❌ Cannot retry | ✅ Can reflect and improve |
| **Use Case** | Simple Q&A, summarization | Research, planning, tool use |

Let's see this in action!

In [3]:
import time

# Example 1: Direct LLM Call (Simple)
print("="*70)
print("EXAMPLE 1: Direct LLM Call")
print("="*70)

# Use the helper function - uses Holistic AI Bedrock by default
llm = get_chat_model("claude-3-5-sonnet")  # Uses Holistic AI Bedrock (recommended)

question = "What is quantum computing?"
print(f"\n❓ Question: {question}")

start_time = time.time()
response = llm.invoke(question)
elapsed = time.time() - start_time

print(f"\n💬 Response: {response.content}")
print(f"\n⏱️  Time: {elapsed:.2f}s")
print("\n✅ Simple and fast!")
print("❌ But... can't use tools, can't reason through steps, single response only")

EXAMPLE 1: Direct LLM Call

❓ Question: What is quantum computing?

💬 Response: Quantum computing is a type of computing that uses quantum mechanics principles to perform calculations and process information. Unlike traditional computers that use bits (0s and 1s), quantum computers use quantum bits or "qubits" that can exist in multiple states simultaneously due to a phenomenon called superposition.

Key features of quantum computing include:

1. Superposition: Qubits can be in multiple states at once
2. Entanglement: Qubits can be interconnected in ways that classical bits cannot
3. Interference: Quantum states can be manipulated to enhance correct solutions

Potential applications include:

1. Complex problem-solving
2. Cryptography and security
3. Drug discovery and molecular modeling
4. Financial modeling
5. Climate change simulation
6. Optimization problems

While quantum computers are still in early development, they have the potential to solve certain problems exponentially fast

### Visual Comparison

#### Direct LLM Call Flow
```
User Question → LLM → Response 
```
- ⚡ Fast and simple
- ❌ Can't use tools, no multi-step reasoning

#### ReAct Agent Flow  
```
User Question → LLM (with tools bound)
                    ↓
          Need to use a tool?
                ↙        ↘
              YES        NO
               ↓          ↓
        Execute Tool   Final Answer
               ↓
        Get Results
               ↓
        Loop back to LLM ←┘
```
- ✅ Can use tools (search, APIs, databases)
- ✅ Multi-step reasoning and self-correction
- ✅ Full observability of the process

## Step 3: Create Your First ReAct Agent

Now let's use **LangGraph's prebuilt `create_react_agent`** function. This is the official implementation that handles:
- State management (message history)
- Tool binding (native function calling)
- Execution loop (reasoning → acting → observing)
- Error handling

In [4]:
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage

# Example 2: ReAct Agent (no tools yet)
print("="*70)
print("EXAMPLE 2: ReAct Agent (No Tools)")
print("="*70)

# Create LLM using helper function
llm = get_chat_model("claude-3-5-sonnet")

# Create ReAct agent with NO tools (for now)
agent = create_react_agent(llm, tools=[])

# Use the same question from Example 1
question = "What is quantum computing?"
print(f"\n❓ Same Question: {question}")

start_time = time.time()
result = agent.invoke({"messages": [HumanMessage(content=question)]})
elapsed = time.time() - start_time

print(f"\n💬 Response: {result['messages'][-1].content}")
print(f"\n⏱️  Time: {elapsed:.2f}s")
print(f"📊 Messages in conversation: {len(result['messages'])}")
print("\n✅ Agent can maintain context, use tools (when provided), and handle multi-turn!")

EXAMPLE 2: ReAct Agent (No Tools)

❓ Same Question: What is quantum computing?

💬 Response: Quantum computing is a type of computing that uses quantum mechanics principles to process information. Unlike traditional computers that use bits (0s and 1s), quantum computers use quantum bits or "qubits" that can exist in multiple states simultaneously due to a phenomenon called superposition.

Key aspects of quantum computing include:

1. Superposition: Qubits can represent multiple states at once, allowing for parallel processing

2. Entanglement: Qubits can be linked together in ways that classical bits cannot, enabling complex calculations

3. Speed: Quantum computers can solve certain problems exponentially faster than classical computers

4. Applications: Potential uses include:
- Cryptography
- Drug discovery
- Climate modeling
- Financial modeling
- Optimization problems

Current challenges:

1. Maintaining qubit stability
2. Error correction
3. Scaling up systems
4. Cost and complexi

### What Just Happened?

The agent:
1. Received your question as a `HumanMessage`
2. LLM processed it (no tools needed for this simple question)
3. Generated an `AIMessage` with the answer
4. Returned the full conversation state

**Key difference from direct LLM call**: The agent maintains state and can loop through tool calls if needed.

## Step 4: Understanding Valyu Search Tool

Before adding tools to the agent, let's understand what the Valyu search tool does:

**Valyu** is an AI-powered search engine that:
- Searches across proprietary sources + web
- Returns structured results with title, URL, content, relevance scores
- Optimized for AI agents (not just humans)

Let's test it directly first.

### Official Valyu Integration

We'll use the official `langchain-valyu` package which provides:
- **ValyuSearchTool**: Search tool compatible with LangChain agents
- **ValyuRetriever**: For RAG (Retrieval-Augmented Generation) pipelines

Documentation: https://python.langchain.com/docs/integrations/providers/valyu/

In [5]:
# Import official Valyu tool from langchain-valyu package
from langchain_valyu import ValyuSearchTool

# Create search tool with configuration
search_tool = ValyuSearchTool(
    valyu_api_key=os.getenv("VALYU_API_KEY"),
    # Optional: configure search parameters (can also be set per-call)
    # search_type="all",  # Search both proprietary and web sources
    # max_num_results=5,   # Limit results
    # relevance_threshold=0.5,  # Minimum relevance score
    # max_price=20.0  # Maximum cost in dollars
)

# Test it directly
print("🔍 Testing Valyu Search Tool Directly")
print("="*70)
test_query = "latest developments in quantum computing"
print(f"Query: {test_query}\n")

# Call the search tool
search_results = search_tool._run(
    query=test_query,
    search_type="all",
    max_num_results=5
)

# Display results (truncated for readability)
result_str = str(search_results)
print("📄 Search Results (first 500 chars):")
print("-"*70)
print(result_str[:500] + "..." if len(result_str) > 500 else result_str)
print("-"*70)
print(f"\n📊 Total data returned: {len(result_str)} characters")
print("✅ This data will be passed to the agent to answer questions!")

🔍 Testing Valyu Search Tool Directly
Query: latest developments in quantum computing

📄 Search Results (first 500 chars):
----------------------------------------------------------------------
{
  "success": true,
  "error": "",
  "tx_id": "tx_8924ad79-e661-41b3-af15-a0797cfb0016",
  "query": "latest developments in quantum computing",
  "results": [
    {
      "title": "News - Quantum Computing Report",
      "url": "https://quantumcomputingreport.com/news/?utm_source=valyu.ai&utm_medium=referral",
      "content": "# News\n\nRecent news items published within the last 6 months on quantum computing developments are listed below. Click on the hyperlinked item to go to the press relea...
----------------------------------------------------------------------

📊 Total data returned: 94641 characters
✅ This data will be passed to the agent to answer questions!


## Step 5: The Power of Tools - WITH vs WITHOUT Search

Now let's see the real magic! We'll ask the same question to:
1. **Agent WITHOUT search** - Will give generic/outdated answer
2. **Agent WITH search** - Will use real-time web data

This demonstrates why tools are critical for ReAct agents.

In [6]:
# Test question about recent events (requires current information)
recent_question = "What are the latest breakthroughs in quantum computing in 2025?"

print("="*70)
print("TEST 1: Agent WITHOUT Search")
print("="*70)
print(f"❓ Question: {recent_question}\n")

start_time = time.time()
result_no_search = agent.invoke({
    "messages": [HumanMessage(content=recent_question)]
})
elapsed_no_search = time.time() - start_time

print("💬 Response (NO search):")
print("-"*70)
response_text = result_no_search['messages'][-1].content
print(response_text[:400] + "..." if len(response_text) > 400 else response_text)
print("-"*70)
print(f"⏱️  Time: {elapsed_no_search:.2f}s")
print(f"\n⚠️  Note: Generic answer, likely outdated or vague (LLM training data cutoff)")

TEST 1: Agent WITHOUT Search
❓ Question: What are the latest breakthroughs in quantum computing in 2025?

💬 Response (NO search):
----------------------------------------------------------------------
I need to point out that I cannot provide information about quantum computing breakthroughs in 2025, as I have a knowledge cutoff date and cannot access real-time or future information. I aim to be transparent about what I do and don't know. I'd be happy to discuss known quantum computing developments up to my last update, or we could focus on the general trajectory of the field based on verified ...
----------------------------------------------------------------------
⏱️  Time: 3.05s

⚠️  Note: Generic answer, likely outdated or vague (LLM training data cutoff)


In [7]:
# Now create agent WITH search tool
print("\n" + "="*70)
print("TEST 2: Agent WITH Search")
print("="*70)

# Create new agent with Valyu search tool (uses Holistic AI Bedrock by default)
llm_with_tools = get_chat_model("claude-3-5-sonnet")
agent_with_search = create_react_agent(
    llm_with_tools,
    tools=[search_tool]  # Add the search tool!
)

print(f"\n❓ Same Question: {recent_question}\n")

start_time = time.time()
result_with_search = agent_with_search.invoke({
    "messages": [HumanMessage(content=recent_question)]
})
elapsed_with_search = time.time() - start_time

print("💬 Response (WITH search):")
print("-"*70)
response_text = result_with_search['messages'][-1].content
print(response_text[:600] + "..." if len(response_text) > 600 else response_text)
print("-"*70)
print(f"⏱️  Time: {elapsed_with_search:.2f}s")
print(f"\n✅ Note: Real-time data, specific sources, up-to-date information!")


TEST 2: Agent WITH Search

❓ Same Question: What are the latest breakthroughs in quantum computing in 2025?

💬 Response (WITH search):
----------------------------------------------------------------------
Based on the search results, here are the key quantum computing breakthroughs and developments in 2025:

1. Microsoft's Majorana 1 Processor:
- Introduced in February 2025
- Designed to scale to a million qubits using topological qubits
- Uses hardware-protected qubits and a novel topological architecture
- Aims to provide a clear path toward scalable quantum computing

2. Google's Willow Chip:
- Demonstrated quantum supremacy for specific computational tasks
- Achieved exponential reduction in errors as more qubits are added
- Performed computations in minutes that would take classical superc...
----------------------------------------------------------------------
⏱️  Time: 27.74s

✅ Note: Real-time data, specific sources, up-to-date information!


### What Happened Under the Hood?

When you gave the agent search capabilities:

1. **LLM received the question** and analyzed it
2. **Decided it needed current info** (can't answer from training data)
3. **Called the Valyu search tool** with structured arguments
4. **Received search results** with current 2025 information
5. **Synthesized the answer** using the retrieved data

This is the **ReAct pattern** in action:
- **Re**ason: "I need current information"
- **Act**: Call search tool
- **Observe**: Get results
- **Respond**: Synthesize answer

## Step 6: Inspect the Message Trace

Let's look at the conversation history to see the ReAct loop in action.

In [8]:
print("📊 Message Trace (WITH search):")
print("="*70)

for i, msg in enumerate(result_with_search['messages']):
    msg_type = type(msg).__name__
    print(f"\n{i+1}. {msg_type}")
    print("-" * 70)
    
    if msg_type == "HumanMessage":
        print(f"   User: {msg.content[:100]}...")
    
    elif msg_type == "AIMessage":
        if hasattr(msg, 'tool_calls') and msg.tool_calls:
            print(f"   AI decided to call tool: {msg.tool_calls[0]['name']}")
            print(f"   Args: {msg.tool_calls[0]['args']}")
        else:
            content_preview = msg.content[:150] + "..." if len(msg.content) > 150 else msg.content
            print(f"   AI response: {content_preview}")
    
    elif msg_type == "ToolMessage":
        content_preview = msg.content[:100] + "..." if len(msg.content) > 100 else msg.content
        print(f"   Tool returned: {content_preview}")

print("\n" + "="*70)
print(f"Total messages: {len(result_with_search['messages'])}")
print("\nThis shows the complete ReAct loop: User → AI (tool call) → Tool → AI (answer)")

📊 Message Trace (WITH search):

1. HumanMessage
----------------------------------------------------------------------
   User: What are the latest breakthroughs in quantum computing in 2025?...

2. AIMessage
----------------------------------------------------------------------
   AI decided to call tool: valyu_deep_search
   Args: {'query': 'major breakthroughs developments quantum computing 2025', 'start_date': '2025-01-01', 'end_date': '2025-12-31', 'max_num_results': 10}

3. ToolMessage
----------------------------------------------------------------------
   Tool returned: {
  "success": true,
  "error": "",
  "tx_id": "tx_62ff553e-5d04-4f1e-9740-a723225f1247",
  "query":...

4. AIMessage
----------------------------------------------------------------------
   AI response: Based on the search results, here are the key quantum computing breakthroughs and developments in 2025:

1. Microsoft's Majorana 1 Processor:
- Introd...

Total messages: 4

This shows the complete ReAct loop:

## Step 7: Try Multi-Turn Conversation

ReAct agents maintain state, so you can have multi-turn conversations!

In [9]:
print("💬 Multi-Turn Conversation Example")
print("="*70)

# Start a new conversation
conversation = {
    "messages": [
        HumanMessage(content="Search for the latest AI agent frameworks in 2025")
    ]
}

# Turn 1
print("\n👤 Turn 1: Search for the latest AI agent frameworks in 2025")
result1 = agent_with_search.invoke(conversation)
print(f"🤖 Agent: {result1['messages'][-1].content[:200]}...\n")

# Turn 2 - follow up question (agent has context from turn 1)
result1['messages'].append(
    HumanMessage(content="Which one is best for production use?")
)
print("👤 Turn 2: Which one is best for production use?")
result2 = agent_with_search.invoke(result1)
print(f"🤖 Agent: {result2['messages'][-1].content[:200]}...\n")

print("="*70)
print("✅ The agent remembers context from previous turns!")
print(f"Total messages in conversation: {len(result2['messages'])}")

💬 Multi-Turn Conversation Example

👤 Turn 1: Search for the latest AI agent frameworks in 2025
🤖 Agent: Based on the search results, here are the key AI agent frameworks that are leading the field in 2025:

1. LangChain
- Most widely adopted framework with over 110k GitHub stars
- Provides modular desig...

👤 Turn 2: Which one is best for production use?
🤖 Agent: Based on the search results, for production use in 2025, Microsoft Agent Framework and LangChain emerge as the most production-ready options, with different strengths:

Microsoft Agent Framework is be...

✅ The agent remembers context from previous turns!
Total messages in conversation: 6


## Summary

Congratulations! You've learned:

### Key Concepts

1. **Direct LLM vs ReAct Agents**
   - Direct LLM: Fast, simple, single-step
   - ReAct Agent: Multi-step reasoning, tool use, self-correction

2. **Modern ReAct Pattern**
   - Uses function calling (not text parsing like original 2022 paper)
   - Structured tool execution via LLM APIs
   - Message-based state management

3. **LangGraph's `create_react_agent`**
   - Official prebuilt implementation
   - Handles state, tools, execution loop
   - Production-ready pattern

4. **The Power of Tools**
   - Agents without tools: Limited to training data
   - Agents with tools: Access to real-time information
   - Valyu search: AI-optimized search engine

### When to Use ReAct Agents

✅ **Use ReAct agents when you need:**
- Real-time information (web search, APIs)
- Multi-step reasoning (planning, breaking down tasks)
- Tool execution (databases, calculators, APIs)
- Self-correction and iteration

❌ **Use direct LLM calls when:**
- Simple Q&A (no external data needed)
- Text generation, summarization
- Speed is critical
- Task is single-step

---

## Next Steps

Continue your learning:

1. **02_custom_tools.ipynb** - Build your own custom tools
2. **03_structured_output.ipynb** - Get JSON responses with Pydantic schemas
3. **04_model_monitoring.ipynb** - Track costs, tokens, and carbon footprint

---

### Resources

- LangGraph Docs: https://langchain-ai.github.io/langgraph/
- ReAct Paper (2022): https://arxiv.org/abs/2210.03629
- Valyu API: https://valyu.ai

---

**Pro Tip**: Always start with the simplest approach (direct LLM) and add complexity (agents, tools) only when needed. The best code is the simplest code that works!